Import methods from API

In [ ]:
from BioTranslator import setup_config, train_text_encoder, get_ontology_embeddings, train_biotranslator, test_biotranslator

Train text encoder

In [ ]:
model_path = './Codebase/TextEncoder/model/text_encoder.pth'
graphine_repo = './Codebase/TextEncoder/data/Graphine/dataset/'
train_text_encoder(graphine_repo, model_path)

Build configs

In [ ]:
import os
def create_repo(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Warning: We created the repo: {}'.format(path))
seq_repo = f'./Codebase/Protein'
create_repo(seq_repo)
seq_config = {
    'task': 'few_shot',
    'max_length': 2000,
    'data_repo': f'{seq_repo}/data/',
    'dataset': 'GOA_Human',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 1500,
    'features': 'seqs, description, network',
    'lr': 0.0003,
    'epoch': 30,
    'batch_size': 32,
    'gpu_ids': '0',
}
vec_repo = f'./Codebase/SingleCell'
vec_config = {
    'task': 'cross-dataset',
    'eval_dataset': 'muris_facs',
    'vec_ontology_repo': f'{vec_repo}/data/Ontology_data/',
    'data_repo': f'{seq_repo}/data/sc_data/',
    'dataset': 'muris_droplet',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 30,
    'lr': 0.0001,
    'epoch': 15,
    'batch_size': 128,
    'gpu_ids': '0',
}

graph_repo = f'./Codebase/Pathway'
graph_config = {
    'max_length': 2000,
    'eval_dataset': 'KEGG',
    'graph_excludes': ['Reactome', 'KEGG', 'PharmGKB'],
    'data_repo': f'{seq_repo}/data/',
    'dataset': 'GOA_Human',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 1500,
    'features': 'seqs, description, network',
    'lr': 0.0003,
    'epoch': 30,
    'batch_size': 32,
    'gpu_ids': '0',
}

seq_config = setup_config(seq_config, 'seq')
vec_config = setup_config(vec_config, 'vec')
graph_config = setup_config(graph_config, 'graph')

cfgs = [seq_config, vec_config, graph_config]

Get Ontology Embeddings

In [ ]:
data_dir = seq_config.data_repo + seq_config.dataset + '/'
text_embs = []
for cfg in cfgs:
    get_ontology_embeddings(model_path, data_dir, cfg)

Train BioTranslators

In [ ]:
encoders = train_biotranslator(cfgs)

Test BioTranslators

In [ ]:
tasks = dict(
    seq = ['prot_func_pred'],
    vec = ['cell_type_cls'],
    graph = ['node_cls', 'edge_pred'])
anno_file = dict(
    seq = [''],
    vec = [''],
    graph = ['', ''],
)
test_biotranslator(data_dir, anno_file, cfgs, encoders, tasks)

In [ ]:
from BioTranslator.utils import load_obj

pathway_dir = './Codebase/Pathway/data/ProteinDataset/KEGG/'
model_path = './Codebase/Pathway/working_space/model/BioTranslator_KEGG.pth'
pathway_graph = load_obj(pathway_dir + 'pathway_graph.pkl')

In [ ]:
import torch
model = torch.load(model_path)
model.annotate(files, cfg, pathway_dir, task='node_cls')

In [ ]:
result_path = './Codebase/Protein/data/GOA_Human/blast_preds_fold_0.pkl'
origin_path = './Codebase/Protein/data/GOA_Human/validation_data_fold_0.pkl'
a = load_obj(result_path)
origin = load_obj(origin_path)

In [ ]:
print(origin)